In [ ]:
#Params
datasetPath = "../datasets/"
dataPath = "./data/"
resultPath = "./results/"
outputFilename="heatmapComparison1"

names = c("Apache-0001","Apache-0002","Apache-0003","Apache-0004","Apache-0005")

In [ ]:
library(ggplot2)
library(plyr)
library(reshape2)
library(tools)

library(gridExtra)
library(grid)

processData <- function(data){

  #Adding a helper colum
  data$index <- 1
  data$index <- ave(data$index, cumsum(c(F, diff(data$sr) > 0)), FUN=seq_along)
  data$freq<-ave( as.numeric(data[[1]]), data[["sr"]] , FUN=length)
  data$index= data$index * 100 / data$freq
  
  #CALCULATE METRICS
  data$accuracy = (data$TP+data$TN)/(data$TP+data$TN+data$FN+data$FP)
  data$specificity=data$TN/(data$TN+data$FP)
  data$precision=data$TP/(data$TP + data$FP)
  data$recall = data$TP / (data$TP+ data$FN) # Recall of class 1
  data$npv=data$TN/(data$TN+data$FN)
  data$negNPV=data$npv-(data$index/100)
  data$negAccuracy=data$accuracy-(data$index/100)
  
  #This melting is because the data wasn't in the proper csv style
    
  data<-melt(data, id.vars = c("FN","FP","TN","TP","sr","t","index","freq"))
    
  return(data)
}

processDataDiffNormalized <- function(data1, data2){
    
  #Adding a helper colum
  data1$index <- 1
  data1$index <- ave(data1$index, cumsum(c(F, diff(data1$sr) > 0)), FUN=seq_along)
  data1$freq<-ave( as.numeric(data1[[1]]), data1[["sr"]] , FUN=length)
  data1$index= data1$index * 100 / data1$freq
  
  #CALCULATE METRICS
  data1$accuracy = (data1$TP+data1$TN)/(data1$TP+data1$TN+data1$FN+data1$FP)
  data1$specificity=data1$TN/(data1$TN+data1$FP)
  data1$precision=data1$TP/(data1$TP + data1$FP)
  data1$recall = data1$TP / (data1$TP+ data1$FN) # Recall of class 1
  data1$npv=data1$TN/(data1$TN+data1$FN)
  data1$negNPV=data1$npv-(data1$index/100)
  data1$negAccuracy=data1$accuracy-(data1$index/100)
    
    
  #Adding a helper colum
  data2$index <- 1
  data2$index <- ave(data2$index, cumsum(c(F, diff(data2$sr) > 0)), FUN=seq_along)
  data2$freq<-ave( as.numeric(data2[[1]]), data2[["sr"]] , FUN=length)
  data2$index= data2$index * 100 / data2$freq
  
  #CALCULATE METRICS
  data2$accuracy = (data2$TP+data2$TN)/(data2$TP+data2$TN+data2$FN+data2$FP)
  data2$specificity=data2$TN/(data2$TN+data2$FP)
  data2$precision=data2$TP/(data2$TP + data2$FP)
  data2$recall = data2$TP / (data2$TP+ data2$FN) # Recall of class 1
  data2$npv=data2$TN/(data2$TN+data2$FN)
  data2$negNPV=data2$npv-(data2$index/100)
  data2$negAccuracy=data2$accuracy-(data2$index/100)
    
    
  #Calculate diff
  accuracy <- data.frame( data1 = data1$accuracy , data2 = data2$accuracy)
  accuracy$accuracy <- (accuracy$data1 - accuracy$data2)
    
  specificity <- data.frame( data1 = data1$specificity , data2 = data2$specificity)
  specificity$specificity <- (specificity$data1 - specificity$data2)
    
  precision <- data.frame( data1 = data1$precision , data2 = data2$precision)
  precision$precision <- (precision$data1 - precision$data2)
    
  recall <- data.frame( data1 = data1$recall , data2 = data2$recall)
  recall$recall <- (recall$data1 - recall$data2)
    
  npv <- data.frame( data1 = data1$npv , data2 = data2$npv)
  npv$npv <- (npv$data1 - npv$data2)
    
  negNPV <- data.frame( data1 = data1$negNPV , data2 = data2$negNPV)
  negNPV$negNPV <- (negNPV$data1 - negNPV$data2)
    
  negAccuracy <- data.frame( data1 = data1$negAccuracy , data2 = data2$negAccuracy)
  negAccuracy$negAccuracy <- (negAccuracy$data1 - negAccuracy$data2)
    
  data <- data.frame(
      accuracy = accuracy$accuracy,
      specificity = specificity$specificity,
      precision = precision$precision,
      recall = recall$recall,
      npv = npv$npv,
      negNPV = negNPV$negNPV,
      negAccuracy = negAccuracy$negAccuracy
  )
  
  #Normalization
  data$accuracy = data$accuracy/abs(data$accuracy);
  data$specificity = data$specificity/abs(data$specificity)
  data$precision = data$precision/abs(data$precision)
  data$recall = data$recall/abs(data$recall)
  data$npv = data$npv/abs(data$npv)
  data$negNPV = data$negNPV/abs(data$negNPV)
  data$negAccuracy = data$negAccuracy/abs(data$negAccuracy)
    
  #This melting is because the data wasn't in the proper csv style
  data<-melt(data)
    #rapply( data["value"], f=function(x) ifelse(is.nan(x),0,x), how="replace" )
        
    is.nan.data.frame <- function(x)
    do.call(cbind, lapply(x, is.nan))

    data[is.nan(data)] <- 0
        
  return(data)
}
           
processDataDiff <- function(data1, data2){
    
  #Adding a helper colum
  data1$index <- 1
  data1$index <- ave(data1$index, cumsum(c(F, diff(data1$sr) > 0)), FUN=seq_along)
  data1$freq<-ave( as.numeric(data1[[1]]), data1[["sr"]] , FUN=length)
  data1$index= data1$index * 100 / data1$freq
  
  #CALCULATE METRICS
  data1$accuracy = (data1$TP+data1$TN)/(data1$TP+data1$TN+data1$FN+data1$FP)
  data1$specificity=data1$TN/(data1$TN+data1$FP)
  data1$precision=data1$TP/(data1$TP + data1$FP)
  data1$recall = data1$TP / (data1$TP+ data1$FN) # Recall of class 1
  data1$npv=data1$TN/(data1$TN+data1$FN)
  data1$negNPV=data1$npv-(data1$index/100)
  data1$negAccuracy=data1$accuracy-(data1$index/100)
    
    
  #Adding a helper colum
  data2$index <- 1
  data2$index <- ave(data2$index, cumsum(c(F, diff(data2$sr) > 0)), FUN=seq_along)
  data2$freq<-ave( as.numeric(data2[[1]]), data2[["sr"]] , FUN=length)
  data2$index= data2$index * 100 / data2$freq
  
  #CALCULATE METRICS
  data2$accuracy = (data2$TP+data2$TN)/(data2$TP+data2$TN+data2$FN+data2$FP)
  data2$specificity=data2$TN/(data2$TN+data2$FP)
  data2$precision=data2$TP/(data2$TP + data2$FP)
  data2$recall = data2$TP / (data2$TP+ data2$FN) # Recall of class 1
  data2$npv=data2$TN/(data2$TN+data2$FN)
  data2$negNPV=data2$npv-(data2$index/100)
  data2$negAccuracy=data2$accuracy-(data2$index/100)
    
    
  #Calculate diff
  accuracy <- data.frame( data1 = data1$accuracy , data2 = data2$accuracy)
  accuracy$accuracy <- (accuracy$data1 - accuracy$data2)
    
  specificity <- data.frame( data1 = data1$specificity , data2 = data2$specificity)
  specificity$specificity <- (specificity$data1 - specificity$data2)
    
  precision <- data.frame( data1 = data1$precision , data2 = data2$precision)
  precision$precision <- (precision$data1 - precision$data2)
    
  recall <- data.frame( data1 = data1$recall , data2 = data2$recall)
  recall$recall <- (recall$data1 - recall$data2)
    
  npv <- data.frame( data1 = data1$npv , data2 = data2$npv)
  npv$npv <- (npv$data1 - npv$data2)
    
  negNPV <- data.frame( data1 = data1$negNPV , data2 = data2$negNPV)
  negNPV$negNPV <- (negNPV$data1 - negNPV$data2)
    
  negAccuracy <- data.frame( data1 = data1$negAccuracy , data2 = data2$negAccuracy)
  negAccuracy$negAccuracy <- (negAccuracy$data1 - negAccuracy$data2)
    
  data <- data.frame(
      accuracy = accuracy$accuracy,
      specificity = specificity$specificity,
      precision = precision$precision,
      recall = recall$recall,
      npv = npv$npv,
      negNPV = negNPV$negNPV,
      negAccuracy = negAccuracy$negAccuracy
  )
    
  #This melting is because the data wasn't in the proper csv style
  data<-melt(data)
    
    rapply( data, f=function(x) ifelse(is.nan(x),0,x), how="replace" )
    
  return(data)
}

In [ ]:
df = data.frame(read.table(paste(dataPath,"results-list.csv",sep=""), header=T,sep=","))
data = list()
for(f in names){
    data[[f]] = read.table(paste(dataPath, f, ".csv", sep=""), header=T,sep=",");
}

#print(processDataDiffNormalized(data[["Apache-0001-classification"]],data[["Apache-0002-classification"]]))
lengthNames = length(names)
#values = c()
values = list()

n1 = 1;
i = 1

#Building the matrix to organize the table of heatmaps
#For each row
for(f1 in names){
    
    n2 = 1;
    base <- processData(data[[f1]])
    
    if(n1==2)
        oldRow = row
    row=c()
    
    #For each column
    for(f2 in names){
        
        #Instead a comparing two same files, show the configuration
        if(f1 == f2){
            row[[n2]] = i
            
            rowData = df[df$results==paste(dataPath,f1,".csv",sep=""),]
            
            #Colors the type of decision tree
            if(rowData$criterion=="entropy" || rowData$criterion=="gini"){
                typeTree="Classification"
                colorTypeTree="blue"
            }else{
                typeTree="Regression"
                colorTypeTree="green"
            }
            
            
            text=""
            text = paste(text, "File : ", dataPath,f1,".csv", "\n",sep="")
            text = paste(text, "Criterion : ", rowData$criterion, "\n",sep="")
            text = paste(text, "Max depth : ", rowData$max_depth, "\n",sep="")
            text = paste(text, "Max features : ", rowData$max_features, "\n",sep="")
            text = paste(text, "Max leaf nodes : ", rowData$max_leaf_nodes, "\n",sep="")
            text = paste(text, "Min samples leaf : ", rowData$min_samples_leaf, "\n",sep="")
            text = paste(text, "Min samples split : ", rowData$min_samples_split, "\n",sep="")
            text = paste(text, "Class weight : ", rowData$class_weight, "\n",sep="")
            text = paste(text, "Min impurity split : ", rowData$min_impurity_split, "\n",sep="")
            text = paste(text, "Min weight fraction leaf : ", rowData$min_weight_fraction_leaf, "\n",sep="")
            text = paste(text, "Presort : ", rowData$presort, "\n",sep="")
            
            #For each metric, save the text element for configuration
            for (v in unique(base$variable)){
                
                values[[v]][[i]] = grid.arrange(arrangeGrob(
                    textGrob(typeTree,gp = gpar(fontsize = 3, col=colorTypeTree)),
                    textGrob(text,gp = gpar(fontsize = 2))
                ))
                
            }
        }else{
            row[[n2]] = i
            diff = processDataDiffNormalized(data[[f1]],data[[f2]])
            diff$sr = base$sr
            diff$index = base$index   
            
            #For each metric, save the heatmap element
            for (v in unique(diff$variable)){
                datatmp=diff[which(diff$variable == v),]
                
                values[[v]][[i]] = ggplot(datatmp,aes(sr,index))+geom_tile(aes(fill=value))+
                  scale_fill_gradient(low = "yellow",high = "red")+
                    xlab("Number of configurations in the training set") +
                    ylab("Percentage of acceptable configurations")+
                    theme(strip.background = element_blank(), legend.position="none")+labs(fill="Normalized\nClassification\nMetric")+ 
                    ggtitle(paste("C",n1," - C",n2,sep=""))+
                    theme(axis.text = element_text(size = rel(0.25)),axis.title= element_text(size = rel(0.25)), plot.title = element_text(face="bold", size=5, hjust=0.5))
            }
            
        }
        i = i+1
        n2 = n2+1;
    }
    
    if(n1==2)
        lay=rbind(oldRow,row)
    else if(n1>2)
        lay=rbind(lay,row)
    
    n1 = n1+1;
}
        
for (v in unique(diff$variable)){
    
    #Save the heatmaps as png
    ggsave(
        grid.arrange(grobs=values[[v]],layout_matrix = lay),
        file=paste(resultPath,outputFilename, "-", v,".png", sep=""),
        width=lengthNames*1,
        height=lengthNames*1.2,
        limitsize = FALSE
    )
    
    #Delete break for having every metrics
    break
}